In [1]:
import os
import torch
import copy

from PIL import Image
from llava.mm_utils import get_model_name_from_path
from llava.model.builder import load_pretrained_model
from llava.mm_utils import process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates

In [2]:
def load_image(img_path):
    
    img = Image.open(img_path)
    return img

def make_heatmap(layer_attn, height_view, width_view):
    
    matrix = layer_attn.view(height_view, width_view)
    matrix = matrix.cpu().numpy()
    return matrix

In [ ]:
qs = "what does it say in the bottom right corner?"
image_path = "/home/FRACTAL/snehan.j/Image-Mines/data/post-letter.jpg"
image = load_image(image_path)

model_path = os.path.expanduser("lmms-lab/llama3-llava-next-8b")
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model("lmms-lab/llama3-llava-next-8b", None, model_name, attn_implementation="eager")

device = "cuda"
image_tensor = process_images([image], image_processor, model.config)
image_tensor = [_image.to(dtype=torch.float16, device=device) for _image in image_tensor]

conv_template = "llava_llama_3"
user_prompt = qs

question = DEFAULT_IMAGE_TOKEN + "\n {}".format(user_prompt) + "\n Answer briefly in only one sentence"

conv = copy.deepcopy(conv_templates[conv_template])
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
prompt_question = conv.get_prompt()

input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
image_sizes = [image.size]

In [ ]:
with torch.inference_mode():
    config, output_ids = model.generate(
        input_ids,
        images=image_tensor,
        image_sizes=image_sizes,
        do_sample=True,
        temperature=0.2,
        max_new_tokens=1024,
        use_cache=True,
        return_dict_in_generate=True,
        output_attentions=True,
        include_base_only=False,
        filter_newline=True)
    
outputs = tokenizer.batch_decode(output_ids['sequences'], skip_special_tokens=True)[0]

sub_crop_height = config[-2]
sub_crop_width = config[-1]

cum_token_averaged_attn = torch.zeros([1, sub_crop_height * sub_crop_width], device = "cpu")
img_token_idx = (input_ids == -200).nonzero(as_tuple=True)[1].item()

for output_attn in output_ids['attentions'][1:]:
    stacked_tensors = torch.stack(output_attn).cpu()
    stacked_tensors = stacked_tensors[24:32]
    mean_values = stacked_tensors.mean(dim=0)
    mean_values = mean_values.squeeze()
    average_values = mean_values.mean(dim=0)
    cum_token_averaged_attn += average_values[img_token_idx + 576: img_token_idx + 576 + sub_crop_height * sub_crop_width]

cum_token_averaged_attn = cum_token_averaged_attn/(len(output_ids['attentions'][1:]))
attn_matrix = make_heatmap(cum_token_averaged_attn, sub_crop_height, sub_crop_width)

In [7]:
image_config = attn_matrix[0].shape
image_attn = attn_matrix[0]

bounding_boxes, clusters = get_mines(image_attn, image_config[0], image_config[1], min_patch_for_mine=4, max_image_mines = 3, max_search_radius=min(image_config[0], image_config[1])/2, cluster_thresh_frac=0.2, apply_gaussian_blur=1)
draw_blocks(img, bounding_boxes, image_config, args.output_path, idx, attn_values[idx][1], upscale_crops=True, delta=15, siglip=False, draw_bool=True)

array([[4.4107437e-06, 4.7087669e-06, 1.2536843e-05, ..., 2.1060307e-06,
        1.1126200e-06, 8.1857042e-06],
       [3.2186508e-06, 1.0867913e-05, 7.1724257e-06, ..., 3.4769375e-06,
        3.1391780e-06, 7.7486038e-07],
       [1.7881393e-06, 6.4571700e-06, 2.0861626e-06, ..., 9.3181925e-06,
        1.0927519e-06, 1.4305115e-06],
       ...,
       [5.3187210e-05, 3.9021175e-05, 6.1972934e-04, ..., 5.0123531e-04,
        7.4605145e-05, 5.2948792e-05],
       [6.9340072e-06, 8.5433321e-06, 1.0794401e-04, ..., 3.7650266e-04,
        8.3168346e-04, 2.5212765e-05],
       [1.4265378e-05, 1.0112922e-05, 3.4173328e-05, ..., 6.9920224e-04,
        4.3630600e-05, 5.9386093e-05]], dtype=float32)